# Interdisciplinary Project
by Marina Sommer | 11778902

Workflow is as follows:

In [84]:
# import packages
from urllib.request import urlretrieve
import os
import re 
import random
import gzip
import json
from collections import Counter
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [136]:
# 1) select crawl: CC-MAIN-2023-40 and download cluster.idx
crawl_name = "CC-MAIN-2023-40"
top_lvl_domain = "at"   # oder "de"
path1 = 'https://data.commoncrawl.org/cc-index/collections/'
path2 = '/indexes/'
path_ccrawl = path1 + crawl_name + path2
url = path_ccrawl + 'cluster.idx'
print(url)

https://data.commoncrawl.org/cc-index/collections/CC-MAIN-2023-40/indexes/cluster.idx


In [137]:
crawl_dir = os.path.join("S:", "msommer", crawl_name, top_lvl_domain)   # evt tld hinzufuegen!! ToDo !!
if not os.path.exists(crawl_dir):
   os.makedirs(crawl_dir)
cluster_file = os.path.join(crawl_dir, "cluster.txt")
#urlretrieve(url, cluster_file)

In [138]:
# 2) filter cdx files with tld '^at,' or '^de,'
regex = '^' + top_lvl_domain + ','
with open(cluster_file, "rt") as file:     
    cdx_files = []
    for line in file:
        tld = line.split("\t")
        match = re.search(regex, tld[0])  # ^at,
        if match: 
            if not tld[1] in cdx_files:
                cdx_files.append(tld[1])
print(cdx_files)      

['cdx-00002.gz', 'cdx-00003.gz', 'cdx-00004.gz']


In [139]:
# 3) randomly choose cdx files, e.g. cdx-00003.gz -> way too much for de
if len(cdx_files) > 3:
    idx = random.sample(range(0, len(cdx_files)), 3)
    print(idx)
    cdx_files = [cdx_files[i] for i in idx]
    print(cdx_files)

In [140]:
# 4) download cdx files (one takes ~ 2 min)
# wget “https://data.commoncrawl.org/cc-index/collections/CC-MAIN-2022-05/indexes/$i.gz"
for file in cdx_files:
    print(file)
    url = path_ccrawl + file
    filename = os.path.join(crawl_dir, file)
    #urlretrieve(url, filename)

cdx-00002.gz
cdx-00003.gz
cdx-00004.gz


In [80]:
from urllib.request import urlretrieve
def tryDownload(url, filename, retries=0):
    if retries > 10: 
        print("Download failed.")
        return
    try:
        urlretrieve(url, filename)  
    except:
        time.sleep(1)
        tryDownload(url, filename, retries+1)

In [145]:
# 5) gunzip and open it - one line looks like the following:
warc_files = []
for file in cdx_files:
    filename = os.path.join(crawl_dir, file)
    print(filename)
    with gzip.open(filename, 'rt') as f:
        for line in f:          
            match = re.search(regex, line)  # ^at,
            if match: 
                l = line.split("{\"url\":")
                string = "{\"url\":" + l[1]
                d = json.loads(string)
                warc = d["filename"]
                if int(d["length"]) > 10000: 
                    warc_files.append(warc)
        count_dict = Counter(warc_files)    # 1 - 73 occurrences per warc file
        
        # only download warc files with a lot of occurrences --> maybe download not necessary? --> see below
        for key, value in count_dict.items():
            if value >= 50:
                # 6) get filename, convert to wet file and download (WARC: ~ 2 min, WET: 15 sec )
                # warc: wget https://data.commoncrawl.org/crawl-data/CC-MAIN-2023-40/segments/1695233511424.48/warc/CC-MAIN-20231004220037-20231005010037-00086.warc.gz
                # wet:  wget https://data.commoncrawl.org/crawl-data/CC-MAIN-2023-40/segments/1695233511424.48/wet/CC-MAIN-20231004220037-20231005010037-00086.warc.wet.gz
                key_path = key.split("/")[-1].replace("warc.gz", "warc.wet.gz")
                filename = os.path.join(crawl_dir, key_path)
                print(filename)
                url = "https://data.commoncrawl.org/" + key
                print(url)
                #tryDownload(url, filename)  
                break
    break

S:msommer\CC-MAIN-2023-40\at\cdx-00002.gz
S:msommer\CC-MAIN-2023-40\at\CC-MAIN-20230923130827-20230923160827-00403.warc.wet.gz
https://data.commoncrawl.org/crawl-data/CC-MAIN-2023-40/segments/1695233506481.17/warc/CC-MAIN-20230923130827-20230923160827-00403.warc.gz


In [99]:
filename = "S:msommer\CC-MAIN-2023-40\CC-MAIN-20230923130827-20230923160827-00876.warc.wet.gz"
with gzip.open(filename, 'rt') as f:
    #file_content = f.read()
    for line in f:          
        print(line)
        #break

WARC/1.0

WARC-Type: warcinfo

WARC-Date: 2023-09-23T13:08:27Z

WARC-Record-ID: <urn:uuid:4e922815-3cac-4e86-94f7-88e752a3270b>

Content-Length: 500

Content-Type: application/warc-fields

WARC-Filename: CC-MAIN-20230923130827-20230923160827-00876.warc.gz



isPartOf: CC-MAIN-2023-40

publisher: Common Crawl

description: Wide crawl of the web for September/October 2023

operator: Common Crawl Admin (info@commoncrawl.org)

hostname: ip-10-67-67-168

software: Apache Nutch 1.19 (modified, https://github.com/commoncrawl/nutch/)

robots: checked via crawler-commons 1.5-SNAPSHOT (https://github.com/crawler-commons/crawler-commons)

format: WARC File Format 1.1

conformsTo: https://iipc.github.io/warc-specifications/specifications/warc-format/warc-1.1/





WARC/1.0

WARC-Type: request

WARC-Date: 2023-09-23T13:18:37Z

WARC-Record-ID: <urn:uuid:86ec99e8-953b-408f-80d1-6cc30ce2127f>

Content-Length: 308

Content-Type: application/http; msgtype=request

WARC-Warcinfo-ID: <urn:uuid:4e922815-3c

UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 1303: character maps to <undefined>

In [21]:
line = 'at,0-niedermayer)/arteriosklvitaminc.htm 20230923150943 {"url": "https://0-niedermayer.at/ArteriosklVitaminC.htm", "mime": "text/html", "mime-detected": "text/html", "status": "200", "digest": "3BUQ2YTS4WFMBHJ3CED2D2ITPJNN6YXF", "length": "14084", "offset": "89255240", "filename": "crawl-data/CC-MAIN-2023-40/segments/1695233506481.17/warc/CC-MAIN-20230923130827-20230923160827-00130.warc.gz", "charset": "UTF-16LE", "languages": "deu"}'

In [ ]:
### QUESTION: How to extract text content of each file?! 
### QUESTION: Wie löse ich das Problem, dass oft nur Überschriften im Text vorkommen?! 

In [124]:
from warcio import ArchiveIterator
import requests
warc_url = url
wet_url = warc_url.replace('/warc/', '/wet/').replace('warc.gz', 'warc.wet.gz')
r = requests.get(warc_url, stream=True)
records = ArchiveIterator(r.raw)

#The first record is a warcinfo record describing the crawl, and all the following requests are conversion records containing the plain text of each response.

record = next(records)
assert record.rec_type == 'warcinfo'

In [135]:
# skip the warcinfo
record = next(records)
# This shows the source page, WARC-Target-URI and other metadata
record.rec_headers.headers
text = record.content_stream().read()
print(text.decode('utf-8'))

<html mip="">
<head>
<meta charset="utf-8" />
<title>正在加载</title>
<meta name="keywords" content="404 Not Found"/>
<meta name="description" content="404 Not Found" />
<script>
(function(){
		var canonicalURL, curProtocol;
		//Get the  tag
		var x=document.getElementsByTagName("link");
		//Find the last canonical URL
		if(x.length > 0){
			for (i=0;i<x.length;i++){
				if(x[i].rel.toLowerCase() == 'canonical' && x[i].href){
					canonicalURL=x[i].href;
				}
			}
		}
		//Get protocol
	    if (!canonicalURL){
	    	curProtocol = window.location.protocol.split(':')[0];
	    }
	    else{
	    	curProtocol = canonicalURL.split(':')[0];
	    }
	    //Get current URL if the canonical URL does not exist
	    if (!canonicalURL) canonicalURL = window.location.href;
	    //Assign script content. Replace current URL with the canonical URL
    	!function(){var e=/([http|https]:\/\/[a-zA-Z0-9\_\.]+\.baidu\.com)/gi,r=canonicalURL,t=document.referrer;if(!e.test(r)){var n=(String(curProtocol).toLowerCas

WARC-Identified-Content-Language: eng
Content-Type: text/plain
Content-Length: 16184

In case of "warc" (https://pypi.org/project/warc/) it should be
record['WARC-Target-URI']
or for "warcio" (https://pypi.org/project/warcio/)
record.rec_headers.get_header('WARC-Target-URI')

(1) use the WET file and read it from the beginning until the WARC-Target-URI is found
(2) use the WARC file together with offset and length and parse the HTML

The warc library has a bug with its gzip handling which causes warc fail to read the entire WET file. To overcome the bug, you should use Python's gzip library to decompress the file stream on the fly as below:

In [ ]:
import gzip
import warc
gzip_fobj = gzip.open(wet_file, "r")
warc_fobj = warc.WARCFile(fileobj=gzip_fobj, compress=False)

evt. filtern nach content-language, content-type und content-length (zb mindestens 1000 zeichen

In [ ]:

import gzip
import warc
gzip_fobj = gzip.open(wet_file, "r")
warc_fobj = warc.WARCFile(fileobj=gzip_fobj, compress=False)

https://stackoverflow.com/questions/34612691/how-can-one-extract-every-payload-from-warc-wet-gz

The code extracts the first 100 text "payloads" from the .warc.wet.gz file. The 100 created text files have shortened names which correspond to their associated url in the warc.wet.gz file. Many of the urls are too long to be used as as convenient names. The file wet_list_out.txt is generated and provides the correspondence between the shortened .txt file names and the associated urls.
The code works. Of course you need to replace warc_wet_gz with the full path name of your own .warc.wet.gz file. If you want all the payloads, not just the first 100, set num to a very large number. Variable num was tacilty assumed to be 100 in the initial post. Why was it rated as not useful? The problem is solved. If you find issues with it please make specific suggestions. 

In [ ]:
import requests
from warcio.archiveiterator import ArchiveIterator
from requests_file import FileAdapter
def print_f_records(fpath,f_out,num):
    record_count = 0
    conversion_count = 0
    header_count = 0
    splitter = "://"
    mx = num
    s = requests.session()
    s.mount('file://',FileAdapter())
    resp = s.get(fpath, stream=True)
    for record in ArchiveIterator(resp.raw, arc2warc=True):
        record_count +=1
        if record.rec_type == 'warcinfo':
            print(record.raw_stream.read())
        elif record.rec_type == 'conversion':
            conversion_count +=1
            if record.raw_stream.read is not None:
                if record.rec_headers.get_header('Content-Type') == 'text/plain':
                    header_count +=1
                    if header_count > mx:
                        continue
                    print("\n text/plain header no: ", header_count)
                    prefix = f"a{header_count}_"        # filenames will begin with a<header_count>
                    fname1 = (record.rec_headers.get_header('WARC-Target-URI'))
                    fname = fname1.split(splitter)[1]   # get the name part which follows ://
                    fname = prefix + fname[:22] + '.txt'    # keep entire file name to 32 characters or less
                    fname = fname.replace("/", "_")
                    content = record.content_stream().read()
                    f = open(fname, 'wb')               # note the wb; write in binay mode because the content is characters
                    f.write(content)
                    f.close
                    line = f'{fname1},{fname}' + "\n" # keep a record of the url translations
                    f_out.write(line)
                    print("created text file: ", fname)
                    if header_count == mx:
                        print(f"\n... completed {mx} txt files ... \n")
    print("Number of records: ", record_count)
    print("Number of conversion types: ", conversion_count)
    print("Number of text/plain http_headers: ", header_count)
    print("\nFINISHED")
warc_wet_gz ='file:////home/psl/CCrawl/text_extraction/text_from_wet/CC-MAIN-20230928063033-20230928093033-00593.warc.wet.gz'
print(f'\n warc.wet.gz file is {warc_wet_gz}')
num=100
f_out=open("wet_list_out.txt", 'w')
print_f_records(warc_wet_gz, f_out, num)
f_out.close()